### Midiendo la edad biológica

Los científicos definen dos tipos principales de edad: la *edad cronológica* y la *edad biológica*. La edad cronológica se basa en cuánto tiempo has estado vivo (años desde el nacimiento), mientras que la edad biológica es una estimación aproximada de la salud de tu cuerpo al medir diferentes biomarcadores.
Una forma de determinar la edad biológica es medir los niveles de metilación en tu genoma. En sitios conocidos como sitios CpG, a lo largo de tu ADN, varias proteínas pueden agregar o quitar grupos metilo para controlar qué genes se expresan en proteínas y cuáles no. A medida que envejeces, sin embargo, los sistemas responsables de mantener este control del genoma comienzan a deteriorarse, lo que provoca errores en la metilación.

El Dr. Steve Horvath, investigador de longevidad en la UCLA, utilizó datos de metilación de ADN para desarrollar un sistema conocido como el **reloj de envejecimiento de Horvath** (Horvath clock), que es capaz de generar una estimación precisa de tu edad biológica. Por ejemplo, después de analizar los niveles de metilación en una muestra de tus células, el reloj podría indicarte que tu edad biológica es de alrededor de 30 años, lo que básicamente significa que tus niveles de metilación (y la salud de tu cuerpo) son similares a los de una persona promedio de 30 años. Si tu edad real está alrededor de los 40 años, esto significa que llevas un estilo de vida saludable, mientras que si solo tienes 20 años, esto sugiere que llevas un estilo de vida poco saludable. Esta información proporciona a las personas el conocimiento necesario para tomar el control de su estilo de vida y vivir de manera saludable.

En este notebook vamos a intentar replicar el reloj epigenético desarrollado por Horvath, utilizando datos de metilación tomados de muestras de sangre (X) para predecir la edad de un individuo (y).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn import linear_model

In [ ]:
#Importar los datos de metilación de pacientes sanos usando Pandas
path = "Healthy_Methylation_Dataset.csv"
healthy_df = pd.read_csv(path)
healthy_df.head()

### Pre-procesamiento de los datos


In [ ]:
# Shuffle dataframe to randomize data order, possibly preventing confounding factors
healthy_df = shuffle(healthy_df)

# Remove patient ID column
healthy_df = healthy_df.drop(['...1'], axis=1)

# Drop all rows with NaN values
healthy_df = healthy_df.dropna()

# Reset Index
healthy_df.reset_index(inplace=True, drop=True)

healthy_df.head()

In [ ]:
#Mapa de calor

import seaborn as sb
C_mat = healthy_df.corr()

fig = plt.figure(figsize = (8,8))

sb.heatmap(C_mat, vmax=1, square=True)
plt.show()

### Normalización y division de los datos en conjuntos de entrenamiento y prueba

El escalado de datos es una práctica recomendada en la mayoría de los casos para mejorar el rendimiento y la estabilidad de los algoritmos de aprendizaje automático, incluyendo SVM.

In [ ]:
# Normalizing the methylation and sex data with a Standard Scaler.
X = healthy_df[['cg09809672', 'cg22736354', 'cg02228185', 'cg01820374', 'cg06493994', 'cg19761273', 'sex']]

# Separating X vs. y dataframes
std_scaler = StandardScaler()
X_std = pd.DataFrame(std_scaler.fit_transform(X), columns=X.columns)
y = healthy_df['age']

In [ ]:
# Separating dataset into train and test subsets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

### Entrenamiento de un modelo de regresion lineal

In [ ]:
# Training the multivariate linear regression model
linear_regressor = linear_model.LinearRegression()
linear_regressor.fit(X_train, y_train)


### Evaluación del modelo de regresion lineal

In [ ]:
# Evaluación en el conjunto de prueba (coeficiente de determinación (R-cuadrado))
print(linear_regressor.score(X_test, y_test))

In [ ]:
predictions = linear_regressor.predict(X_test)

# Since age cannot be negative, changing all negative predictions to age 0
for n, element in enumerate(predictions):
    if element < 0:
        predictions[n] = 0

# Looking at sample predictions for the testing set
for i in range(0, 10):
    print("Predicción:", predictions[i], "\tEdad real:", y_test.iloc[i])

Veamos si podemos mejorar la capacidad predictiva del modelo usando SVM.

### Maquinas de Soporte Vectorial para Regresión (SVR)

Similar a las SVM, las SVR utilizan el concepto de un hiperplano y un margen, pero existen diferencias en sus definiciones. En SVR, el margen se define como una región de tolerancia al error del modelo, que también se llama "tubo ε-insensible".  Esto crea un equilibrio entre la complejidad del modelo y su capacidad de generalización.


In [ ]:
#Entrenamiendo de un modelo SVR
from sklearn.svm import SVR

svr = SVR(kernel='rbf', epsilon=0.1, gamma= 'scale', C=100.0)
svr.fit(X_train, y_train)

In [ ]:
#Evaluación del modelo SVR
svr.score(X_test, y_test)

In [ ]:
predictions = svr.predict(X_test)

# Since age cannot be negative, changing all negative predictions to age 0
for n, element in enumerate(predictions):
    if element < 0:
        predictions[n] = 0

# Looking at sample predictions for the testing set
for i in range(0, 10):
  print("Prediccion:", predictions[i],"\tEdad real:", y_test.iloc[i])

Como vemos, el reloj SVR predice un poco mejor la edad de individuos sanos.

A continuación se muestra un grafico de residuos que permite evaluar la calidad del modelo de regresión SVR.

In [ ]:
from yellowbrick.regressor import ResidualsPlot
visualizer = ResidualsPlot(svr, title="Gráfico de residuos para Edad Real vs. Predicha en el cohorte de pacientes sanos")

visualizer.score(X_test, y_test)
visualizer.show()

### Ajuste de los hiperparámetros

In [ ]:
from sklearn.model_selection import GridSearchCV

#Ajuste de hiperparametro: kernel, epsilon y C mediante Validación Cruzada

# Define la rejilla de hiperparámetros a buscar
param_grid = {'kernel': ['rbf', 'linear'],
    'epsilon': [0.1, 0.2, 0.5, 0.7, 1.0],
    'C': [1, 10, 100]
}

# Crea un objeto GridSearchCV
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')

# Ajusta el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Imprime la mejor combinación de hiperparámetros encontrada
grid_search.best_params_

In [ ]:
#Reentrenamos el modelo usando el mejor set de hiperparámetros
svr = SVR(kernel=grid_search.best_params_['kernel'], epsilon=grid_search.best_params_['epsilon'], gamma= 'scale', C=grid_search.best_params_['C'])
svr.fit(X_train, y_train)
svr.score(X_test, y_test)


### Prueba en pacientes enfermos

El modelo SVR es capaz de predecir con relativa precisión la edad de individuos sanos basándose en sus niveles de metilación.

Entrenamos el modelo con datos de indivudos sanos porque se supone que en este cohorte la edad biológica no debería diferir demasiado de su edad cronológica.

Sin embargo, es de esperar que personas enfermas muestren niveles de metilación correspondientes a una peor edad biológica. Para probar esta suposicion, importamos un conjunto de datos de alrededor de 627 individuos enfermos (después de eliminar los valores faltantes) y utilizamos el modelo SVR pre-entrenado para comparar la edad biológica predicha con su edad cronológica real.

In [ ]:
#Importar los datos de metilación de pacientes enfermos usando Pandas
path = "Disease_Methylation_Dataset.csv"
disease_df = pd.read_csv(path)
disease_df.head()

In [ ]:
# Remove patient ID column
disease_df = disease_df.drop(['Unnamed: 0'], axis=1)

# Drop all rows with NaN values
disease_df = disease_df.dropna()

disease_df.head()

In [ ]:
disease_X = disease_df[['cg09809672', 'cg22736354', 'cg02228185', 'cg01820374', 'cg06493994', 'cg19761273', 'sex']]
disease_X = pd.DataFrame(std_scaler.fit_transform(disease_X), columns=disease_X.columns)
#se aplico el mismo escalado de datos que antes para poder comparar las predicciones
disease_Y = disease_df['age']

In [ ]:
disease_predictions = svr.predict(disease_X)

In [ ]:
plt.plot(disease_predictions)
plt.plot(disease_Y)
plt.title(" Edad real vs. predicha en pacientes enfermos")
plt.ylabel('Edad')
plt.legend(['Edad predicha', 'Edad real'], loc='upper left')
plt.show()